# 0) Data Load/Fix 

In [7]:
import datetime
import pandas as pd 

In [3]:
cd data/ 

/Users/yennanliu/sequence_predictor/data


In [4]:
ls

AMZN.csv                           annual-barley-yields-per-acre.csv
APPLE.csv                          annual-common-stock-price-us.csv
FB.csv                             quarterly-sp-500.csv
SNAP.csv


In [2]:
# help function

def timestamp_fix(x):
    x_ = datetime.datetime.strptime(x, "%d/%m/%Y").strftime("%Y-%m-%d")
    return x_
  

def timestamp_fix_(x):
    # fix mm/dd/yy -> yyyy-mm-dd
    y_=x.split('/')
    mm,dd,yy = y_[0], y_[1],y_[2]
    yy = '20'+yy
    y_output = yy+'/'+ mm +'/'+ dd
    return y_output

In [1]:
######## data timestamp form   ########

In [71]:
"""
d1 = "22.05.15"
y='04/12/18'
#datetime.datetime.strptime(d1,'%d.%m.%Y').strftime('%Y-%d-%m')
y_=y.split('/')
dd,mm,yy = y_[0], y_[1],y_[2]
yy = '20'+yy
y_output = yy+'/'+ mm +'/'+ dd
y_output
"""

'\nd1 = "22.05.15"\ny=\'04/12/18\'\n#datetime.datetime.strptime(d1,\'%d.%m.%Y\').strftime(\'%Y-%d-%m\')\ny_=y.split(\'/\')\ndd,mm,yy = y_[0], y_[1],y_[2]\nyy = \'20\'+yy\ny_output = yy+\'/\'+ mm +\'/\'+ dd\ny_output\n'

In [83]:
df_SNAP= pd.read_csv('SNAP.csv')

In [72]:
# example 
#datetime.datetime.strptime("04/12/2018", "%d/%m/%Y").strftime("%Y-%m-%d")

In [84]:
df_SNAP['Date'] = df_SNAP.Date.map(timestamp_fix_)

In [85]:
df_SNAP['Date'] = pd.to_datetime(df_SNAP['Date'])

In [88]:
df_SNAP.columns

Index(['Date', ' Open', ' High', ' Low', ' Close', ' Volume'], dtype='object')

In [73]:
#df_apple.Date.ix[0]

In [86]:
df_SNAP.to_csv('SNAP.csv')

In [3]:
######## fix column name  ########

In [5]:
ls data

AMZN.csv                           annual-barley-yields-per-acre.csv
APPLE.csv                          annual-common-stock-price-us.csv
FB.csv                             quarterly-sp-500.csv
SNAP.csv


In [31]:
df_SNAP= pd.read_csv('data/SNAP.csv')

In [32]:
df_SNAP.columns

Index(['Unnamed: 0', 'Date', ' Open', ' High', ' Low', ' Close', ' Volume'], dtype='object')

In [21]:
col = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']

In [36]:
df_SNAP[col].reset_index(drop=True).head(3)

,Date,Open,High,Low,Close,Volume
0,2018-04-12,15.24,15.50,14.77,14.92,21276192
1,2018-04-11,14.44,14.98,14.42,14.80,17711174
2,2018-04-10,14.29,14.58,14.18,14.48,16351816


In [24]:
#df_AMZN
#df_APPLE

In [34]:
df_SNAP.to_csv('SNAP.csv')

# 1) Analysis/EDA

# 2) RNN 

In [89]:
import tensorflow as tf
from tensorflow.contrib import rnn

/Users/yennanliu/anaconda3/envs/ds_dash/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


In [96]:
# help function 

def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, timesteps, 1)

    # Define a lstm cell with tensorflow
    lstm_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)

    # Get lstm cell output
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

In [114]:
# import our own data 
from data_prepare import *
df_fb = load_data('FB')
col = ['Date', ' Open', ' High', ' Low', ' Close', ' Volume']
df_fb = df_fb[col]
df_fb.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']

/Users/yennanliu/sequence_predictor/data/FB.csv
   Unnamed: 0        Date    Open    High     Low   Close    Volume
0           0  2018-04-12  166.98  167.45  163.10  163.87  38262956
1           1  2018-04-11  165.36  168.65  163.25  166.32  56144633
2           2  2018-04-10  157.93  165.98  157.01  165.04  58947041
3           3  2018-04-09  157.82  160.53  156.04  157.93  34915227
4           4  2018-04-06  157.73  161.42  156.81  157.20  41644812


In [118]:
#df_fb.Low

In [120]:
df_fb.head(3)

,Date,Open,High,Low,Close,Volume
0,2018-04-12,166.98,167.45,163.10,163.87,38262956
1,2018-04-11,165.36,168.65,163.25,166.32,56144633
2,2018-04-10,157.93,165.98,157.01,165.04,58947041


In [95]:
# Training Parameters
learning_rate = 0.001
training_steps = 10000
batch_size = 128
display_step = 200

# Network Parameters
num_input = 28 # MNIST data input (img shape: 28*28)
timesteps = 28 # timesteps
num_hidden = 128 # hidden layer num of features
num_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
X = tf.placeholder("float", [None, timesteps, num_input])
Y = tf.placeholder("float", [None, num_classes])

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}


In [97]:
logits = RNN(X, weights, biases)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [ ]:
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, 3+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, timesteps, num_input))
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

In [98]:

# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, training_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, timesteps, num_input))
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for 128 mnist test images
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, timesteps, num_input))
    test_label = mnist.test.labels[:test_len]
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: test_data, Y: test_label}))
    
    

<tf.Operation 'init' type=NoOp>